In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

In [2]:
city_data = pd.read_csv('output_data/cities.csv')

city_data.set_index("City ID", inplace=True)

city_data.head()

,City,Country,Latitude,Longitude,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date
City ID,,,,,,,,,
0,Acapulco de Juárez,MX,16.8634,-99.8901,80.40,62,0,6.42,1644015202
1,Tuktoyaktuk,CA,69.4541,-133.0374,-23.80,76,0,18.41,1644015172
2,Vanimo,PG,-2.6741,141.3028,81.73,77,70,4.88,1644015421
3,Saint George,US,37.1041,-113.5841,51.94,22,0,3.00,1644015267
4,Provideniya,RU,64.3833,-173.3000,3.63,59,0,12.93,1644015421


In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
#Set up heatmap
fig = gmaps.figure()
locations = city_data[["Latitude", "Longitude"]]
fig.add_layer(gmaps.heatmap_layer(locations))

In [5]:
#Set the Lat and Lng as locations and Humidity as the weight
locations = city_data[["Latitude", "Longitude"]]
weight = city_data["Humidity (%)"]

figure_layout = {
    'width': '500px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

#Add heatmap layer to map
fig = gmaps.figure(layout=figure_layout)
heatmap_layer = gmaps.heatmap_layer(locations, weight, dissipating = False, max_intensity = 50, point_radius = 1)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…

In [14]:
#Narrow down data frame to ideal vacation conditions
vacay_temp_min = 75.0
vacay_temp_max = 87.5
wind_speed_max = 10.0
humidity_max = 75
cloud_max = 20.0

#Create data frame from only the ideal vacation locations
vacation_cities = city_data[(city_data["Max Temp (F)"] >= vacay_temp_min) & 
                            (city_data["Max Temp (F)"] < vacay_temp_max) &
                            (city_data["Wind Speed (mph)"] <= wind_speed_max) &
                            (city_data["Humidity (%)"] < humidity_max) &
                            (city_data["Cloudiness (%)"] < cloud_max)                           
                           ]

#Confirm that the "ideal" parameters are not too narrow
len(vacation_cities.index)

16

## Hotel Map

In [7]:
# Find closest hotel/lodging

#Create a data frame that will include hotel inforamtion
hotel_df = vacation_cities.copy()

#Add empty columns for "Hotel Name" and "Hotel Rating" to data frame
hotel_df["Hotel Name"] = ""
hotel_df["Hotel Rating"] = ""

# Base url for gmaps API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Set up parameter dictionary. Note: "lodging" is listed as a "Place Type" in gmaps API documentation, "hotel" is not
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

#Loop through cities in the hotel_df data frame to look for hotels
for index, row in hotel_df.iterrows():

    #Use the city's latitude and longitude to find the nearest hotel/lodging
    city_lat = row["Latitude"]
    city_lon = row["Longitude"]

    #append lat, lon to list of parameters
    params["location"] = f"{city_lat},{city_lon}"
    
    #get data from gmaps
    response = requests.get(base_url, params=params).json()  

    try:       
        #Add first hotel result for each city to data frame        
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        hotel_df.loc[index, "Hotel Rating"] = response["results"][0]["rating"]
        print(f"Retrieving results for city {index}: {row[0]}")
        
    except (KeyError, IndexError):
        print(f"Missing field. Skipping {row[0]}.")

#Drop data frame rows with blanks
hotel_df = hotel_df[(hotel_df["Hotel Name"] != '') & (hotel_df["Hotel Rating"] != '')]   

#Convert ratings from objects to numbers
hotel_df["Hotel Rating"] = pd.to_numeric(hotel_df["Hotel Rating"], errors='coerce')

#Only keep the top 10 rated hotels, ensures that no more than 10 hotels are pinned on the map
hotel_df = hotel_df.nlargest(10, "Hotel Rating", keep='first')

#Display the data frame
hotel_df

Retrieving results for city 0: Acapulco de Juárez
Retrieving results for city 38: Rikitea
Retrieving results for city 135: Kapaa
Retrieving results for city 156: Otukpo
Retrieving results for city 182: Cayenne
Missing field. Skipping Buin.
Retrieving results for city 242: Puerto Escondido
Retrieving results for city 253: Ixtapa
Missing field. Skipping Bafoulabé.
Retrieving results for city 332: ‘Āhuimanu
Missing field. Skipping Nioro.
Retrieving results for city 477: Les Cayes
Retrieving results for city 499: Maturin
Retrieving results for city 526: Maragogi
Retrieving results for city 529: Koutiala
Retrieving results for city 540: Yenagoa
Missing field. Skipping ‘Ewa Beach.


,City,Country,Latitude,Longitude,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date,Hotel Name,Hotel Rating
City ID,,,,,,,,,,,
526,Maragogi,BR,-9.0122,-35.2225,79.52,69,15,9.80,1644015421,Salinas do Maragogi All Inclusive Resort,4.8
0,Acapulco de Juárez,MX,16.8634,-99.8901,80.40,62,0,6.42,1644015202,HS Hotsson Smart Acapulco,4.4
253,Ixtapa,MX,20.7000,-105.2000,75.22,76,7,8.01,1644015201,Hotel Plaza Providencia,4.4
332,‘Āhuimanu,US,21.4447,-157.8378,80.78,58,0,3.00,1644015503,Paradise Bay Resort,4.3
38,Rikitea,PF,-23.1203,-134.9692,78.55,71,10,7.87,1644015166,People ThankYou,4.2
242,Puerto Escondido,MX,15.8500,-97.0667,81.91,60,1,5.32,1644015479,Aldea del Bazar Hotel and Spa,4.2
135,Kapaa,US,22.0752,-159.3190,81.48,59,0,1.99,1644015450,Sheraton Kauai Coconut Beach Resort,4.1
499,Maturin,VE,9.7500,-63.1767,78.28,55,1,9.60,1644015547,Hotel Residencial La Pichincha,4.1
477,Les Cayes,HT,18.2000,-73.7500,76.50,73,14,3.94,1644015541,Villa Orphee,4.0
